In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
#x_train = np.load('path/to/train.npy')

x_train, x_test = x_train/255.0, x_test/255.0

In [3]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train[:48000], y_train[:48000], epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2


/Users/tharundv/miniconda3/envs/myenv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8492 - loss: 0.5249 - val_accuracy: 0.9460 - val_loss: 0.1790
Epoch 2/2
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9621 - loss: 0.1256 - val_accuracy: 0.9581 - val_loss: 0.1403


In [4]:
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.9572 - loss: 0.1339


[0.11423265188932419, 0.9642999768257141]

In [5]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = y_test

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_classes, y_pred_classes))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step
Model Accuracy on Test Set: 96.43%

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.97      0.95      0.96      1032
           3       0.92      0.98      0.95      1010
           4       0.95      0.97      0.96       982
           5       0.98      0.96      0.97       892
           6       0.97      0.98      0.97       958
           7       0.95      0.97      0.96      1028
           8       0.98      0.93      0.95       974
           9       0.96      0.94      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
[[ 965    0    0    2    3    1    4    1    2    2]
 [   0 1120    2    3    0    1    2    6    1    0]
 [   7    3  9

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ayuraj/asl-dataset")

print("Path to dataset files:", path)

100%|██████████| 56.9M/56.9M [00:04<00:00, 13.1MB/s]

Extracting files...


Path to dataset files: /Users/tharundv/.cache/kagglehub/datasets/ayuraj/asl-dataset/versions/1


In [8]:
train_dir = f'{path}/asl_dataset/asl_dataset'
test_dir = f'{path}/asl_dataset/asl_dataset'

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [9]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 2515 images belonging to 36 classes.
Found 2515 images belonging to 36 classes.


In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, epochs=2, validation_data=test_data)

Epoch 1/2


/Users/tharundv/miniconda3/envs/myenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/tharundv/miniconda3/envs/myenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.1381 - loss: 3.1448 - val_accuracy: 0.7785 - val_loss: 0.7278
Epoch 2/2
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.6837 - loss: 1.0296 - val_accuracy: 0.9070 - val_loss: 0.2942


In [11]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))
print(f"Overall Accuracy: {accuracy_score(y_true, y_pred):.4f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9067 - loss: 0.3005
Test Loss: 0.2942
Test Accuracy: 0.9070
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.02      0.01      0.02        70
           1       0.00      0.00      0.00        70
           2       0.05      0.06      0.06        70
           3       0.01      0.01      0.01        70
           4       0.02      0.01      0.01        70
           5       0.03      0.03      0.03        70
           6       0.00      0.00      0.00        70
           7       0.01      0.01      0.01        70
           8       0.01      0.01      0.01        70
           9       0.06      0.06      0.06        70
           a       0.00      0.00      0.00        70
           b       0.00      0.00      0.00        70
           c       0.00      0.00      0.00        70
           d       0.04      0.04      0.04        70
           e

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

movies = [
    {"title": "Movie A", "description": "A thrilling action movie with a hero", "genre": "Action"},
    {"title": "Movie B", "description": "A romantic comedy with a love story", "genre": "Romance"},
    {"title": "Movie C", "description": "A sci-fi adventure with spaceships", "genre": "Sci-Fi"},
    {"title": "Movie D", "description": "A comedy about friends", "genre": "Comedy"},
    {"title": "Movie E", "description": "A thrilling action-packed movie", "genre": "Action"}
]

df = pd.DataFrame(movies)

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df.index[df['title'] == title].tolist()[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [ ]:
movie = input("Enter you movie name : ")
print(f"Recommendations for '{movie}':")
print(get_recommendations(movie))

Enter you movie name : Movie D
Recommendations for 'Movie D':
1    Movie B
0    Movie A
2    Movie C
Name: title, dtype: object
